In [ ]:
import numpy as np
import glob
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr
import pyproj as proj
from scipy.stats import norm
%matplotlib inline

def folder2cube(files, size_raster):
    base_set = glob.glob(files)
    cube = np.zeros(size_raster + (len(base_set),))
    for i, model in enumerate(base_set): 
        cube[:,:,i] = np.loadtxt(model, skiprows = 1).reshape(size_raster)
    return cube, len(base_set)


size_raster = (250,162)


In [ ]:
base_cube, base_n = folder2cube('data/Hackaton/BaseSet/MapSimu__*.data',size_raster)
top_cube, top_n = folder2cube('data/Hackaton/TopSet/MapSimu__*.data',size_raster)


In [ ]:
volume_file = 'data/Hackaton/VolumeDistribution/Volumes'

vol = pd.read_csv(volume_file, delim_whitespace=True)

In [ ]:
ds = xr.Dataset()  

X_corner = 390885
Y_corner = 7156947
dx, dy = 25, 25

#top_model = np.min([np.min(base_cube),np.min(top_cube)])
#bottom_model = np.max([np.max(base_cube),np.max(top_cube)])

top_model = 900
bottom_model = 1100
dz = 100


xx = np.linspace(X_corner, X_corner+size_raster[0]*dx, size_raster[0])
yy = np.linspace(Y_corner, Y_corner+size_raster[1]*dy, size_raster[1])
zz = np.linspace(bottom_model, top_model, dz)

model = np.linspace(0, top_model, base_n)

ds.coords['X'] = xx
ds.coords['Y'] = yy
ds.coords['Z'] = zz
ds.coords['MODEL'] = model

ds['BASE'] = (('X', 'Y', 'MODEL'), base_cube)
ds['TOP'] = (('X', 'Y', 'MODEL'), top_cube)


In [ ]:
base_mean = ds['BASE'].mean(dim='MODEL')
base_std = ds['BASE'].std(dim='MODEL')

top_mean = ds['TOP'].mean(dim='MODEL')
top_std = ds['TOP'].std(dim='MODEL')


In [ ]:
top_cloud = np.zeros(size_raster + (dz,))
base_cloud = np.zeros(size_raster + (dz,))

for x in range(0,len(xx)):
    for y in range(0,len(yy)):
        top_cloud[x,y,:] = norm.pdf(zz,top_mean.values[x,y],top_std.values[x,y])

for x in range(0,len(xx)):
    for y in range(0,len(yy)):
        base_cloud[x,y,:] = norm.pdf(zz,base_mean.values[x,y],base_std.values[x,y])


In [ ]:
np.save('graham.pkl', top_cloud)
top_cloud=[]
top_cloud = np.load('graham.pkl.npy')
top_cloud.shape

In [ ]:
plt.imshow(top_cloud[:,128,:].T)
plt.colorbar()
plt.show()


In [ ]:
plt.imshow(top_cloud[:,:,80].T)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(top_cloud[:,:,35].T)
plt.colorbar()
plt.show()

In [ ]:
import bokeh

from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool
from bokeh.io import output_notebook

output_notebook()

p = figure(x_range=(0, 250), y_range=(0, 162),
           tools=[HoverTool(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])])

# must give a vector of image data for image parameter
p.image(image=[top_cloud[:,:,30]], x=0, y=0, dw=250, dh=162, palette="Spectral11")

show(p)  # open a browser

In [ ]:
top_cloud.shape

In [ ]:
import holoviews as hv
import numpy as np
hv.extension('bokeh')

ds = hv.Dataset((np.arange(0,100,1), np.linspace(0., 162., 162), np.linspace(0., 250., 250),
                top_cloud),
                kdims=['depth', 'y', 'x'],
                vdims=['z'])

type(ds.data), list(ds.data.keys())

In [ ]:
%opts Image(cmap='viridis')
%opts Image [height=400 width=700] 
ds.to(hv.Image, ['x', 'depth']).redim(z=dict(range=(0,0.05))).options(colorbar=True, invert_yaxis=True)

In [ ]:
%opts Image(cmap='viridis')
%opts Image [height=400 width=616] 
ds.to(hv.Image, ['x', 'y']).redim(z=dict(range=(0,0.05))).options(colorbar=True, invert_yaxis=True)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
i, j, k = np.where(top_cloud > 0.04)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(i, j, -k, zdir='z', c=(top_cloud[top_cloud > 0.04]*255), alpha=0.02)
ax.set_xlim3d(0,250)
ax.set_ylim3d(0,162)
ax.set_zlim3d(-100,-0)
plt.figure(figsize=(8,10));